<img src="Dau.png">

#### MASEF, University Paris-Dauphine 2021:   Bryan Delamour 

## Hybrid scheme for the rough Bergomi model

#### Ref: Hybrid scheme for Brownian semistationary processes
Mikkel Bennedsen, Asger Lunde, Mikko S. Pakkanen

In [9]:
import numpy as np 
import scipy.special as sp
import scipy.stats as sps

## Rough Bergomi Model

$$
S_{t}:=S_{0} \exp ( \underbrace{\int_{0}^{t} \sqrt{v_{s}} \mathrm{~d} Z_{s}-\frac{1}{2} \int_{0}^{t} v_{s} \mathrm{~d} s}_{=: X_{t}}), \quad t \in[0, T]
$$

$$
v_{t}:=\xi_{t}^{0} \exp (\eta \underbrace{\sqrt{2 \alpha+1} \int_{0}^{t}(t-s)^{\alpha} \mathrm{d} W_{s}}_{=: Y_{t}}-\frac{\eta^{2}}{2} t^{2 \alpha+1}), \quad t \in[0, T]
$$

$$ \mathrm{~d}Z_s = \rho \mathrm{~d}W_s + \sqrt{1 - \rho^2} \mathrm{~d}B_s$$ 

### Euler Scheme on $X_t$

$$
{X}_{t_{i+1}}={X}_{t_{i}}- \frac{v_{t_i}}{2} \Delta t+\sqrt{{v}_{t_{i}}} \Delta Z_{i}
$$

### Hybrid Scheme for $Y_t$

We define the hybrid scheme to discretize $Y_{t}$, for any $t \geq 0$, as


$$Y_{t}=\int_{0}^{t} g(t-s) \sigma_{s} \mathrm{~d} W_{s}, \quad t \geq 0$$

For the fractionnal Brownian motion we take: 

$$ g(t-s) = (t-s)^\alpha$$
$$ \sigma_s = \sqrt{2\alpha + 1} $$ 
$$ L_g = 1 $$

$$
Y_{t}^{n}:=\check{Y}_{t}^{n}+\hat{Y}_{t}^{n}
$$


where
$$
\begin{array}{l}
\check{Y}_{t}^{n}:=\sum_{k=1}^{\min \{\lfloor n t\rfloor, \kappa\}} L_{g}\left(\frac{k}{n}\right) \sigma_{t-\frac{k}{n}} \int_{t-\frac{k}{n}}^{t-\frac{k}{n}+\frac{1}{n}}(t-s)^{\alpha} \mathrm{d} W_{s} \\
\hat{Y}_{t}^{n}:=\sum_{k=\kappa+1}^{\lfloor n t\rfloor} g\left(\frac{b_{k}}{n}\right) \sigma_{t-\frac{k}{n}}\left(W_{t-\frac{k}{n}+\frac{1}{n}}-W_{t-\frac{k}{n}}\right)
\end{array}
$$

##### Remark 3.1

In the case of the $\mathcal{T B S S}$ process $Y$, the observations $Y_{\frac{i}{n}}^{n}, i=0,1, \ldots,\lfloor n T\rfloor$, given by the hybrid scheme can be computed via

$$
Y_{\frac{i}{n}}^{n}=\sum_{k=1}^{\min \{i, \kappa\}} L_{g}\left(\frac{k}{n}\right) \sigma_{i-k}^{n} W_{i-k, k}^{n}+\sum_{k=\kappa+1}^{i} g\left(\frac{b_{k}^{*}}{n}\right) \sigma_{i-k}^{n} W_{i-k}^{n}
$$



using the random vectors $\mathbf{W}_{i}^{n}, i=0,1, \ldots,\lfloor n T\rfloor-1$, and random variables $\sigma_{i}^{n}$, $i=0,1, \ldots,\lfloor n T\rfloor-1$

$$
\hat{Y}_{\frac{i}{n}}^{n}=\sum_{k=1}^{N_{n}} \Gamma_{k} \Xi_{i-k}=(\Gamma \star \Xi)_{i}
$$
where

$$
\begin{aligned}
\Gamma_{k} &:=\left\{\begin{array}{ll}
0, & k=1, \ldots, \kappa \\
g\left(\frac{b_{k}^{*}}{n}\right), & k=\kappa+1, \kappa+2, \ldots, \lfloor n T\rfloor
\end{array}\right.\\
\Xi_{k}: &=\sigma_{k}^{n} W_{k}^{n}, & k= 0, \ldots,\lfloor n T\rfloor-1
\end{aligned}$$

#### Proposition 2.8

The asymptotic MSE induced by the discretization, is minimized by the sequence $\mathbf{b}^{*}$ given by


$$
b_{k}^{*}=\left(\frac{k^{\alpha+1}-(k-1)^{\alpha+1}}{\alpha+1}\right)^{1 / \alpha}, \quad k \geq \kappa+1
$$

We only implemented the first order Hybrid Scheme, ie, $\kappa = 1$

In [46]:
class rBergomi_h(object):
    
    def __init__(self, paths = 1, N = 1000, T = 1.00, H = 0.5): 
        
        self.paths = paths # nb of samples 
        self.N = N        # discretization 
        self.T = T        # maturity 
        self.H = H        # Hurst parameter
        self.alpha = self.H - 0.5  
    
    def sigma(self, k, j):
        
        if (k == 1) and (j == 1):
            return( 1/self.N )
        
        if (k == 1) or ( j == 1 ):
            if (k > j): 
                return ( ((k-1)**(self.alpha + 1) - (k - 2)**(self.alpha + 1)) / ((self.alpha + 1) * self.N**(self.alpha+1)))
            else : 
                return ( self.sigma(j,k) )
        
        if ( k == j ): 
            return (((k-1)**(2*self.alpha + 1) - (k - 2)**(2*self.alpha + 1)) / ((2*self.alpha + 1) * self.N**(2*self.alpha+1)))
        
        if ( k > j) : 
            return ( ( ((j-1)**(self.alpha + 1)) * ((k-1)**self.alpha) * sp.hyp2f1(-self.alpha, 1, self.alpha + 2, (j-1)/(k-1) ) 
                        - ((j-2)**(self.alpha + 1)) * ((k-2)**self.alpha) * sp.hyp2f1(-self.alpha, 1, self.alpha + 2, (j-2)/(k-2) ) )
                       * ( 1/( (self.alpha+1)*((self.N)**(2*self.alpha+1) )) ) ) 

        else: 
            return (self.sigma(j,k))
        
        
    def dW(self):
        
        vect_sigma = np.vectorize(self.sigma)
        mat = np.zeros( (2 ,2) )
        vec = np.arange(1,3, 1)
        mat[0,:] = vect_sigma(1,vec)
        mat[1,:] = vect_sigma(2,vec)
        return ( np.random.multivariate_normal([0]*(2),mat,(self.paths, int(self.T*self.N))) )


    def W_h(self, dW):# Y TBSS 
        
        W_h = np.zeros((dW.shape[0], int(self.T*self.N)+1) )
        
        Y1 = dW[:,:,1]*np.sqrt(2*self.H)
        
        W_h[:,1]= Y1[:,0]
        
        left = np.arange(1, int(self.T*self.N)+1, 1)
        g_bk = (left[1:]**(self.alpha+1) - left[:-1]**(self.alpha+1)) / (self.alpha+1)

        Gamma = np.zeros(int(self.T*self.N))
        Xi = np.zeros((dW.shape[0], int(self.T*self.N)))
        
        Gamma[1: ] = g_bk/ (self.N**self.alpha)
        Xi = np.sqrt(2*self.H)*dW[:,:,0]
        
        Y2 = np.zeros((dW.shape[0],int(self.T*self.N)))
        for i in range(dW.shape[0]): 
            Y2[i,1:int(self.T*self.N)] = np.convolve(Gamma,Xi[i,:])[1:int(self.T*self.N)]     
        
        W_h[:, 2:] = Y1[:,1:] + Y2[:,1:int(self.T*self.N)]

        return(W_h)
    
    
    def V( self, V_0, nu, W_h): #Variance rBergomi
        var = (np.arange(0, int(self.T*self.N)+1, 1)/(self.N) )**(2*self.H)
        Vt = V_0*np.exp(nu*W_h-(nu**2)*var*0.5 )

        return (Vt )
      
    def S(self, S0, rho, dW, V): # Euler Scheme
        dW2 = np.random.randn(dW.shape[0], int(self.T*self.N))*(1/np.sqrt(self.N))*(np.sqrt(1-rho**2))+dW[:,:,0]*rho
        S_bar_ = np.c_[ S0 + np.zeros(dW.shape[0]), np.ones( (dW.shape[0],int(self.N*self.T) ) )]
        S_bar_[:,1:] = np.cumsum(np.sqrt(V[:,:-1])*dW2 - V[:,:-1]*(1/(2*self.N)), axis=1)
        S_bar_[:,1:] = S0*np.exp(S_bar_[:,1:])      
        return( S_bar_ )
                                
    def S1 (self, S0, rho, dW, V): # for turbocharging monte carlo pricing
        S_bar_ = np.c_[ S0 + np.zeros(dW.shape[0]), np.ones( (dW.shape[0],int(self.N*self.T) ) )]
        S_bar_[:, 1:] = np.cumsum( np.sqrt(V[:,:-1])*dW[:,:,0]*rho-(rho**2)*V[:,:-1]*(1/(2*self.N)), axis=1 )
        S_bar_[:, 1:] = S0*np.exp(S_bar_[:, 1:])
        return (S_bar_ ) 
        